# LLM Tags - Эксперименты

Этот notebook демонстрирует использование библиотеки `llm_tags` для автоматического тегирования обращений клиентов с помощью LLM.

## Что умеет библиотека:
- Тегирование обращений через Ollama (qwen2.5:32b)
- Батчинг (30-100 обращений за раз)
- Многопоточность (5 параллельных запросов)
- Кеширование результатов
- Фильтрация по количеству существующих тегов


In [1]:
# Импорт библиотек
import pandas as pd
import sys
sys.path.insert(0, '/home/alex/tradeML/llm_tags')

from llm_tags import TaggingPipeline, OllamaLLM

print("✓ Библиотеки загружены!")


✓ Библиотеки загружены!


In [2]:
## Пример 1: Базовое тегирование


In [3]:
# Создаем тестовые данные
df = pd.DataFrame({
    "text": [
        "Не могу войти в личный кабинет",
        "Забыл пароль от аккаунта",
        "Товар не пришел уже неделю",
        "Как изменить тариф?",
        "Проблемы с интернетом",
        "Хочу подключить роуминг",
        "Не работает мобильное приложение",
        "Списали деньги дважды",
    ]
})

print(f"Загружено {len(df)} обращений:")
df


Загружено 8 обращений:


,text
0,Не могу войти в личный кабинет
1,Забыл пароль от аккаунта
2,Товар не пришел уже неделю
3,Как изменить тариф?
4,Проблемы с интернетом
5,Хочу подключить роуминг
6,Не работает мобильное приложение
7,Списали деньги дважды


In [4]:
# Создаем pipeline для тегирования
pipeline = TaggingPipeline(
    batch_size=4,  # Обрабатываем по 4 обращения за раз
    num_workers=2  # 2 параллельных потока
)

print("✓ Pipeline создан")


✓ Pipeline создан


In [5]:
# Промпт для тегирования
prompt = """
Проанализируй обращения клиентов телеком-оператора и определи теги для каждого.
Теги должны отражать основную тему обращения.

Возможные категории тегов:
- авторизация: проблемы со входом в систему
- пароль: восстановление или смена пароля
- доставка: вопросы по доставке товаров/сим-карт
- тарифы: вопросы по тарифным планам
- технические_проблемы: неполадки с услугами
- роуминг: вопросы по роумингу
- приложение: проблемы с мобильным приложением
- биллинг: вопросы по оплате и списаниям

Можешь создавать новые теги если нужно.
"""

print("✓ Промпт подготовлен")


✓ Промпт подготовлен


In [6]:
# Запускаем тегирование
# ВАЖНО: Убедитесь что Ollama запущена (ollama serve)
result_df, tags_dict = pipeline.tag(
    df,
    text_column="text",
    tag_prompt=prompt,
    max_tags=3  # Максимум 3 тега на обращение
)

print("\n✓ Тегирование завершено!")


Всего обращений: 8
Для обработки: 8
Пропущено (уже есть 10+ тегов): 0
Батчей для обработки: 2
Батч 1/2 обработан
Батч 2/2 обработан
Обработка завершена. Всего тегов в словаре: 0

✓ Тегирование завершено!


In [7]:
# Смотрим результаты
print("Результаты тегирования:")
result_df[["text", "tags"]]


Результаты тегирования:


,text,tags
0,Не могу войти в личный кабинет,
1,Забыл пароль от аккаунта,
2,Товар не пришел уже неделю,
3,Как изменить тариф?,
4,Проблемы с интернетом,
5,Хочу подключить роуминг,
6,Не работает мобильное приложение,
7,Списали деньги дважды,


In [8]:
# Смотрим словарь тегов с описаниями
print("Словарь тегов:")
for tag, desc in tags_dict.items():
    print(f"  • {tag}: {desc}")


Словарь тегов:


## Пример 2: Использование существующих тегов


In [9]:
# Новая партия обращений
df2 = pd.DataFrame({
    "text": [
        "Не могу зайти в аккаунт",
        "Потерял пароль",
        "Медленный интернет",
        "Хочу сменить тариф на более дешевый",
    ]
})

# Используем теги из предыдущего прогона
result_df2, tags_dict2 = pipeline.tag(
    df2,
    text_column="text",
    tag_prompt=prompt,
    existing_tags=tags_dict,  # Передаем существующие теги
    max_tags=2
)

print("Результаты:")
result_df2[["text", "tags"]]


Всего обращений: 4
Для обработки: 4
Пропущено (уже есть 10+ тегов): 0
Батчей для обработки: 1


KeyboardInterrupt: 

In [ ]:
print(f"Было тегов: {len(tags_dict)}")
print(f"Стало тегов: {len(tags_dict2)}")


## Пример 3: Пропуск уже размеченных обращений


In [ ]:
# Данные с уже проставленными тегами
df3 = pd.DataFrame({
    "text": [
        "Не могу войти",
        "Забыл пароль",
        "Проблемы с интернетом",
        "Новое обращение без тегов",
        "Еще одно новое обращение",
    ],
    "tags": [
        "авторизация",
        "пароль, авторизация",
        "технические_проблемы, интернет",
        "",  # Нет тегов
        "",  # Нет тегов
    ]
})

print("Исходные данные:")
df3


In [ ]:
# Обработать только строки БЕЗ тегов
result_df3, tags_dict3 = pipeline.tag(
    df3,
    text_column="text",
    tag_prompt=prompt,
    skip_if_tags_count=1,  # Пропустить строки с >= 1 тегом
    max_tags=2
)

print("\nРезультаты (обработаны только пустые строки):")
result_df3[["text", "tags"]]


## Пример 4: Загрузка данных из файла


In [ ]:
# Пример загрузки из CSV
# df_from_file = pd.read_csv("your_data.csv")

# Пример загрузки из сжатого CSV
# df_from_file = pd.read_csv("your_data.csv.gz", compression="gzip")

# Пример загрузки из Parquet
# df_from_file = pd.read_parquet("your_data.parquet")

# Пример использования данных из llm_clustering
sample_file = "/home/alex/tradeML/llm_clustering/ai_data/demo_sample.csv"
df_sample = pd.read_csv(sample_file, nrows=5)  # Загружаем только 5 строк для примера

print(f"Загружено {len(df_sample)} обращений из файла")
print(f"Колонки: {list(df_sample.columns)}")
df_sample.head()


## Пример 5: Кастомные настройки


In [ ]:
# Создание кастомного LLM с другими параметрами
custom_llm = OllamaLLM(
    api_url="http://localhost:11434/api",
    model="qwen2.5:32b",  # Можно указать другую модель
    temperature=0.5  # Меньше креативности, больше детерминизма
)

# Создание pipeline с кастомными настройками
custom_pipeline = TaggingPipeline(
    llm=custom_llm,
    batch_size=30,      # Меньше батч
    num_workers=3,      # Меньше потоков
    cache_dir="./my_cache"  # Своя директория для кеша
)

print("✓ Кастомный pipeline создан")


## Полезные советы


In [ ]:
"""
ВАЖНО перед запуском:
1. Убедитесь что Ollama запущена: ollama serve
2. Установите модель: ollama pull qwen2.5:32b

Параметры pipeline.tag():
- text_column: название колонки с текстом (по умолчанию "text")
- tag_prompt: промпт с инструкциями по тегированию
- existing_tags: словарь существующих тегов (опционально)
- skip_if_tags_count: пропустить строки с >= N тегов (по умолчанию 10)
- max_tags: максимум тегов на обращение (по умолчанию 5)

Кеширование:
- Результаты автоматически сохраняются в ./cache/tags_cache.csv
- При повторном запуске уже обработанные обращения берутся из кеша
- Это позволяет безопасно перезапускать процесс

Производительность:
- batch_size: 30-100 (больше = быстрее, но больше нагрузка на LLM)
- num_workers: 3-10 (зависит от мощности вашего сервера)
- temperature: 0.3-0.7 (меньше = более детерминированные результаты)
"""
print("✓ Советы прочитаны")


## Ваши эксперименты

Используйте ячейки ниже для своих экспериментов с библиотекой.


In [ ]:
# Ваш код здесь

